<a href="https://colab.research.google.com/github/Dinke265/NSTIP/blob/main/NSTIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rpy2

In [2]:
%load_ext rpy2.ipython


In [3]:
%%R
print("Hello from R!")

[1] "Hello from R!"


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
%cd /content/drive/My Drive/NSTIP/

/content/drive/My Drive/NSTIP


In [ ]:
%%R
# Install the necessary package (if not already installed)
install.packages("readr")

# Load the readr package
library(readr)

# Example of reading a file from the local file system (after uploading)
allgradsdata <- read_csv("/content/drive/My Drive/NSTIP/AllGraduatesData2022.csv")

# Display the first few rows of the dataset
head(allgradsdata)


In [29]:
%%R
head(allgradsdata)

names(allgradsdata) <- tolower(names(allgradsdata))

print(names(allgradsdata))

 [1] "public institution"  "private institutuin" "programs"           
 [4] "no.of females"       "no.of males"         "level"              
 [7] "stem"                "sti"                 "ict"                
[10] "miscelaaneouss"      "degree_level"       


In [10]:
%%R
# Print the unique categories for the level
print(unique(allgradsdata$level))


 [1] "Degree"                     "Diploma"                   
 [3] "Certificate"                "PhD"                       
 [5] "Masters"                    "Degree Level"              
 [7] "Masters Level"              "Postgraduate Diploma Level"
 [9] NA                           "Undergraduate"             
[11] "Postgraduate"               "Bachelor's degree"         
[13] "diploma"                    "CERTIFICATE"               
[15] "NCC LEVEL 4 DIPLOMA"        "NCC LEVEL 5 DIPLOMA"       
[17] "DEGREE"                     "Level 4"                   
[19] "Level 6"                    "Bachelors"                 
[21] "DIPLOMA"                    "BACHELOR'S"                
[23] "MASTER'S"                   "Doctors"                   
[25] "Post Graduate"              "MASTERS DEGREE"            
[27] "7"                          "9"                         
[29] "10"                         "UNDERGRADUATE"             
[31] "Chichewa Program"           "English Program"    

In [ ]:
%%R

install.packages("tidyverse")
install.packages("dplyr")

library(tidyverse)
library(dplyr)


In [27]:
%%R
#Clean the level variable

allgradsdata <- allgradsdata %>%
  mutate(level = case_when(
    level == "Degree" ~ "Bachelors",
    level == "Degree Level" ~ "Bachelors",
    level == "Undergraduate" ~ "Bachelors",
    level == "BACHELOR'S" ~ "Bachelors",
    level == "DEGREE" ~ "Bachelors",
    level == "Bachelor's degree" ~ "Bachelors",
    level == "Masters Level" ~ "Masters",
    level == "MASTERS DEGREE" ~ "Masters",
    level == "UNDERGRADUATE" ~ "Bachelors",
    level == "MASTER'S" ~ "Masters",
    level == "Diploma Level" ~ "Diploma",
    level == "Foundetion Diploma" ~ "Diploma",
    level == "Diploma Level 5" ~ "Diploma",
    level == "NCC LEVEL 5 DIPLOMA" ~ "Diploma",
    level == "NCC LEVEL 4 DIPLOMA" ~ "Diploma",
    level == "diploma" ~ "Diploma",
    level == "DIPLOMA" ~ "Diploma",
    level == "Level 4" ~ "Diploma",
    level == "Level 6" ~ "Bachelors",
    level == "Chichewa Program" ~ "Certificate",
    level == "English Program" ~ "Certificate",
    level == "CERTIFICATE" ~ "Certificate",
    level == "PhD" ~ "Doctorate",
    level == "Doctors" ~ "Doctorate",
    level == "7" ~ "Bachelors",
    level == "9" ~ "Masters",
    level == "10" ~ "Doctorate",
    level == "4" ~ "Certificate",
    level == "5" ~ "Diploma",
    level == "6" ~ "Diploma",
    level == "Post Graduate" ~ "Masters",
    level == "Postgraduate" ~ "Masters",
    level == "CA Level 1" ~ "Diploma",
    level == "Postgraduate Diploma Level" ~ "Postgraduate Other",
    TRUE ~ as.character(level)
  ))


In [28]:
%%R
print(unique(allgradsdata$level)) # Check the categories for the level have been correctly replaced

[1] "Bachelors"          "Diploma"            "Certificate"       
[4] "Doctorate"          "Masters"            "Postgraduate Other"
[7] NA                  


In [34]:
%%R
#Renaming some variables
allgradsdata <- allgradsdata %>%
  rename(
    males = `no.of males`,
    females = `no.of females`,
    private = `private institutuin`,
    public = `public institution`)

print(names(allgradsdata))

 [1] "public"         "private"        "programs"       "females"       
 [5] "males"          "level"          "stem"           "sti"           
 [9] "ict"            "miscelaaneouss" "degree_level"  


Creating an indicator variable for institution type.
1= public institution
0= private institution

In [36]:
%%R
allgradsdata <- allgradsdata %>%
  mutate(
    institution_type = case_when(
      !is.na(public) ~ 1,
      !is.na(private) ~ 0,
      TRUE ~ NA_integer_
    )
  )

In [37]:
%%R
head(allgradsdata)

# A tibble: 6 × 12
  public private programs   females males level  stem   sti   ict miscelaaneouss
  <chr>  <chr>   <chr>        <dbl> <dbl> <chr> <dbl> <dbl> <dbl>          <dbl>
1 MZUNI  <NA>    Bachelor …       6    11 Bach…    NA    NA    NA             NA
2 MZUNI  <NA>    Bachelor …       8    20 Bach…    NA    NA    NA             NA
3 MZUNI  <NA>    Bachelor …      21    28 Bach…    NA    NA    NA             NA
4 MZUNI  <NA>    Bachelor …      40    52 Bach…    NA    NA    NA             NA
5 MZUNI  <NA>    Diploma i…      11    25 Dipl…    NA    NA    NA             NA
6 MZUNI  <NA>    Diploma i…       4     9 Dipl…    NA    NA    NA             NA
# ℹ 2 more variables: degree_level <chr>, institution_type <dbl>


Numbers of males and females by degree level

In [39]:
%%R
allgradsdata %>%
  group_by(level) %>%
  summarise(
    total_males = sum(males, na.rm = TRUE),
    total_females = sum(females, na.rm = TRUE)
  ) %>%
  print()


# A tibble: 7 × 3
  level              total_males total_females
  <chr>                    <dbl>         <dbl>
1 Bachelors                 4247          3506
2 Certificate                127           120
3 Diploma                   1447          1168
4 Doctorate                    8             1
5 Masters                    351           218
6 Postgraduate Other          16             8
7 <NA>                     13478          9533
